# Treinando uma rede neural no MNIST com Keras

Este exemplo simples demonstra como conectar datasets do TensorFlow (TFDS) a um modelo Keras.


Copyright 2020 Autores do TensorFlow Datasets, licenciado sob Apache License, versão 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/datasets/keras_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/datasets/keras_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/datasets/keras_example.ipynb"> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/datasets/keras_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Passo 1: crie seu pipeline de entrada

Comece construindo um pipeline de entrada eficiente usando recomendações dos documentos a seguir:

- Guia de [Dicas de desempenho](https://www.tensorflow.org/datasets/performances)
- O guia [Desempenho melhor com a API `tf.data`](https://www.tensorflow.org/guide/data_performance#optimize_performance)


### Carregue um dataset

Carregue o dataset MNIST com os seguintes argumentos:

- `shuffle_files=True`: os dados MNIST são apenas armazenados num único arquivo, mas para datasets maiores com vários arquivos no disco, é uma boa prática embaralhá-los durante o treinamento.
- `as_supervised=True`: Retorna uma tupla `(img, label)` em vez de um dicionário `{'image': img, 'label': label}`.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Construa um pipeline de treinamento

Aplique as seguintes transformações:

- `tf.data.Dataset.map`: o TFDS fornece imagens do tipo `tf.uint8`, enquanto o modelo espera `tf.float32`. Portanto, você precisa normalizar as imagens.
- `tf.data.Dataset.cache` à medida que você ajusta o dataset na memória, armazene-o em cache antes de embaralhá-lo para obter um melhor desempenho.<br> **Observação:** transformações aleatórias devem ser aplicadas após o armazenamento em cache.
- `tf.data.Dataset.shuffle`: para obter aleatoriedade verdadeira, defina o buffer de embaralhamento para que tenha o tamanho total do dataset.<br>**Observação:** para datasets grandes que não cabem na memória, use `buffer_size=1000` se o seu sistema permitir.
- `tf.data.Dataset.batch` : organize os elementos do dataset em lote depois de embaralhar para obter lotes exclusivos para cada época.
- `tf.data.Dataset.prefetch`: é uma boa prática finalizar o pipeline fazendo uma pré-busca [por questões de desempenho](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Construa um pipeline de avaliação

Seu pipeline de teste é semelhante ao pipeline de treinamento, com pequenas diferenças:

- Você não precisa chamar `tf.data.Dataset.shuffle`.
- O cache é feito depois da criação do lote porque os lotes podem ser iguais entre épocas.

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Passo 2: crie e treine o modelo

Conecte o pipeline de entrada do TFDS a um modelo Keras simples, compile o modelo e treine-o.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)